<a href="https://colab.research.google.com/github/Ocrabit/Hackathon2024/blob/main/ExampleGraphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install uszipcode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.2 MB/s eta 0:00:00
  Created wheel for atomicwrites: filename=atomicwrites-1.4.1-py2.py3-none-any.whl size=6941 sha256=31eef6974e985556b084f51989f222a92e33892c3780487b2f6c044b8897bc88
  Stored in directory: /root/.cache/pip/wheels/34/07/0b/33b15f68736109f72ea0bb2499521d87312b932620737447a2
Successfully built atomicwrites
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.29
    Uninstalling SQLAlchemy-2.0.29:
      Successfully uninstalled SQLAlchemy-2.0.29
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt

from matplotlib.pyplot import subplots
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
from ISLP import confusion_table
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from ISLP.models import sklearn_sm
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit, cross_val_score)
from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV

import plotly.express as px
import ast
import geopandas as gpd
from uszipcode import SearchEngine

ImportError: cannot import name 'Zipcode' from 'uszipcode' (/usr/local/lib/python3.10/dist-packages/uszipcode/__init__.py)

In [ ]:
df_income_zipcode = pd.read_csv('/content/drive/MyDrive/Hackathons/4 5 24/Datasets/Transportation Data April 5/IRSIncomeByZipCode.csv')
df_rt_bus = pd.read_csv('/content/drive/MyDrive/Hackathons/4 5 24/Datasets/Transportation Data April 5/Regional_Transportation_Authority_Bus_Stops_20240402.csv')
df_weGo_bus = pd.read_csv('/content/drive/MyDrive/Hackathons/4 5 24/Datasets/Transportation Data April 5/WeGo_Transit_Bus_Stops_20240402.csv')
df_SDoH = pd.read_csv('/content/drive/MyDrive/Hackathons/4 5 24/Datasets/Transportation Data April 5/SDoH_TN_zipcode_2015-2020.csv')


In [ ]:
print("init",len(df_weGo_bus))
df_dropedNa = df_weGo_bus.dropna(subset=['Mapped Location'])
print("end",len(df_dropedNa))

data = df_dropedNa['Mapped Location'].str.strip('()').str.split(',').str[:2]
data.head()
geom = pd.DataFrame(data.tolist(), columns=['lat', 'long'], index=df_dropedNa.index) \
         .apply(pd.to_numeric, errors='coerce')
df_filteredLatLon = df_dropedNa.join(geom)

init 2578
end 2524


In [ ]:
len(df_filteredLatLon)

2524

In [ ]:
def getZipCode(lat, lon):
  try:
    search = SearchEngine()
    result = search.by_coordinates(lat=lat,lng=lon, returns=1)
    return result[0].zipcode
  except:
    return 'Nan'

In [ ]:
getZipCode(36.086851,-86.686667)

'37217'

In [ ]:
zipCodes = []
for i,row in df_filteredLatLon.iterrows():
    zipCodes += [getZipCode(row['lat'],row['long'])]

df_filteredLatLon['ZipCodes'] = zipCodes
df_filteredLatLon['ZipCodes'].value_counts()

In [ ]:
df_filteredLatLon['ZipCodes'].value_counts()

37208    232
37203    190
37212    184
37206    166
37211    147
37210    136
37245    135
37228    134
37207    132
37213    117
37216    111
37115     98
37204     97
37201     88
37217     86
37013     74
37214     74
37138     67
37218     66
37205     63
37221     33
37072     22
37130     16
37086     12
37076     11
37167     11
37220      9
37215      8
37189      5
Name: ZipCodes, dtype: int64

In [ ]:
df_filteredLatLon['Route Name'].value_counts()

In [ ]:
df_filteredLatLon.drop(columns=['Mapped Location'])

,Stop ID,Stop Abbreviation,Stop Name,Shelter?,Bench?,Route Numbers,Route Name,lat,long
4,4133,GRWHIWN,GREEN LN & WHITES CREEK PIKE WB,False,False,41,GOLDEN VALLEY,36.236249,-86.816722
5,4083,Z9AEDGSN,_ 9TH AVE S & EDGEHILL AVE SB,False,False,8,8TH AVENUE SOUTH,36.142642,-86.780897
6,5066,MXODONEL,DONELSON/DELL STATION OUTBOUND,True,False,"15,55",MURFREESBORO PIKE,36.105615,-86.672004
7,94,17ADORSN,17TH AVE S & DOROTHY PL SB,False,False,2,BELMONT,36.137623,-86.795609
8,3219,COC14AWN,COCKRILL ST & 14TH AVE N,False,False,42,ST. CECILIA - CUMBERLAND,36.175944,-86.804242
...,...,...,...,...,...,...,...,...,...
2572,5320,EZECONWN,EZELL PIKE CONVENIENCE CTR WB,True,False,52,NOLENSVILLE PK BRT LITE,36.086851,-86.686667
2573,5166,JEF6AVWM,JEFFERSON ST & 6TH AVE WB,False,False,60,MUSIC CITY BLUE CIRCUIT,36.173796,-86.789456
2574,4698,NEEJAPEF,NEELY'S BEND RD & JASPERSON DR EB,False,False,76,MADISON CONNECTOR,36.247943,-86.682633
2576,4714,OLDFORWN,OLD HICKORY BLVD & FOREST PARK RD WB,False,False,76,MADISON CONNECTOR,36.263003,-86.703336


In [ ]:
fig = px.scatter_mapbox(df_filteredLatLon,lat='lat',lon='long',hover_name = "Stop Name", color='Route Name')
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin=dict(l = 0, r = 0, t = 0, b = 0))
fig.show()

In [ ]:
zipTenessee = gpd.read_file('/content/drive/MyDrive/Hackathons/4 5 24/Datasets/map.geojson')

In [ ]:
figGeoJ = px.choropleth_mapbox(df_filteredLatLon, geojson=zipTenessee, locations='Stop Name', color='Route Name',
                           color_continuous_scale="Viridis",
                           #range_color=(50, 100),
                           mapbox_style="carto-positron",
                           zoom=10, center = {"lat": 36.16803, "lon": -86.77787},
                           opacity=0.5
                           )
figGeoJ.show()

In [ ]:
print("init", len(df_rt_bus))
df_rt_bus.dropna(subset=['Mapped Location'])
print("end",len(df_rt_bus))

data = df_rt_bus['Mapped Location'].str.strip('()').str.split(',').str[:2]
geom = pd.DataFrame(data.tolist(), columns=['lat', 'long'], index=df_rt_bus.index) \
         .apply(pd.to_numeric, errors='coerce')
df_filtered_rt = df_rt_bus.join(geom)

init 88
end 88


In [ ]:
for col in df_filtered_rt:
  print(col)

Stop ID Number
Stop Abbreviation
Stop Name
Bench
Shelter
Line Number
Line Name
Mapped Location
lat
long


In [ ]:
df_filtered_rt.drop(columns=['Mapped Location'])

,Stop ID Number,Stop Abbreviation,Stop Name,Bench,Shelter,Line Number,Line Name,lat,long
0,4418,MCC4_20,MUSIC CITY CENTRAL 4TH - BAY 20,False,True,94,CLARKSVILLE EXPRESS,36.166545,-86.781895
1,4422,MCC5_6,MUSIC CITY CENTRAL 5TH - BAY 6,True,True,94,CLARKSVILLE EXPRESS,36.166501,-86.781233
2,4249,21WE,21ST AVE PAST WEST END AVE SB,False,False,87,GALLATIN EXPRESS,36.149489,-86.800523
3,4184,MCSMJ,MUSIC CITY STAR MT. JULIET STATION,True,True,90,MUSIC CITY STAR,36.199912,-86.517904
4,4425,MCC5_8,MUSIC CITY CENTRAL 5TH - BAY 8,False,True,92,HENDERSONVILLE EXPRESS,36.166768,-86.781424
...,...,...,...,...,...,...,...,...,...
83,4185,MCSMS,MUSIC CITY STAR MARTHA'S STATION,True,True,90,MUSIC CITY STAR,36.229824,-86.425658
84,4430,MCC5_7,MUSIC CITY CENTRAL 5TH - BAY 7,True,True,88,DICKSON EXPRESS,36.166804,-86.781446
85,5053,DRCRPR,DRAKES CREEK PARK-N-RIDE,False,False,92,HENDERSONVILLE EXPRESS,36.312976,-86.611232
86,4249,21WE,21ST AVE PAST WEST END AVE SB,False,False,92,HENDERSONVILLE EXPRESS,36.149489,-86.800523


In [ ]:
figrt = px.scatter_mapbox(df_filtered_rt,lat='lat',lon='long',hover_name = "Stop Name", color='Line Name')
figrt.update_layout(mapbox_style="carto-positron")
figrt.update_layout(margin=dict(l = 0, r = 0, t = 0, b = 0))
figrt.show()